In [1]:
import requests as re
import pandas as pd
import datetime
import altair as alt
import altair_latimes as lat

In [2]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Download
Source: [California Water Watch](https://cww.water.ca.gov/), PRISM

Some parameters

In [3]:
today = datetime.date.today()
today

datetime.date(2024, 1, 24)

In [4]:
atlasID="South Coast"
varKey="PPT_YTD"

Query

In [5]:
url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={today}&atlasID={atlasID}&varKey={varKey}'

In [6]:
data = re.get(url).json()['data']

### Parse

In [7]:
data.keys()

dict_keys(['seriesNames', 'series', 'min', 'avg', 'max', 'yearEndAvg'])

In [8]:
prcp_min=data['min']
prcp_min

0.67

In [9]:
prcp_avg=data['avg']
prcp_avg

7.84

In [10]:
prcp_max=data['max']
prcp_max

25.18

In [11]:
year_end_avg = data['yearEndAvg']
year_end_avg

17.48

Build dataframe

In [12]:
columns = data['seriesNames']

In [13]:
columns.insert(0, "date")

In [14]:
df = pd.DataFrame(data['series'], columns=columns)

In [15]:
df.columns = df.columns.str.lower()

In [16]:
df['date'] = pd.to_datetime(df['date'])

In [29]:
df[ df.date == "2024-01-22" ]

,date,current,average
113,2024-01-22,4.58,7.63


### Plot

In [17]:
gauge = df[
    (~df.current.isna())
]

In [18]:
gauge = gauge[gauge.date == gauge.date.max()]

In [19]:
gauge["record"] = prcp_max

In [20]:
gauge["pct_of_avg"] = (gauge["current"] / gauge["average"]) * 100

In [21]:
alt.Chart(
    pd.melt(gauge, id_vars="date", value_vars=["current","average"])
).mark_bar().encode(
    x="value",
    y=alt.Y("variable", sort="-x"),
    color="variable"
).properties(
    title="Year-to-date precipitation in California compared to average",
    width=600,
    height=200
)

alt.Chart(...)

In [22]:
line_chart = pd.melt(df, id_vars="date", value_vars=["current","average"])

In [23]:
alt.Chart(line_chart).mark_line().encode(
    x="date:T",
    y="value",
    color="variable",
).properties(
    title=f"California precipitation in {today.year} compared to average",
    width=600
)

alt.Chart(...)

### Export

In [32]:
df.to_csv("../../data/processed/precipitation/south-coast-timeseries.csv", index=False)